# <center> LAB 9 - Gale-Shapley and Irving Algorithms </center>
<center>Harold Alfredo Díaz Ortiz</center>

# Punto 1

Find the best and worst cases for the Gale-Shapley and Irving ( <a rel="noreferrer noopener" href="http://www.google.com/url?q=http%3A%2F%2Fwww.dcs.gla.ac.uk%2F~pat%2Fjchoco%2Froommates%2Fpapers%2FComp_sdarticle.pdf&amp;sa=D&amp;sntz=1&amp;usg=AFQjCNGQ0dZWRogUukUGh3nfPtxoSf_zKQ" target="_blank">An Efficient Algorithm for the “Stable Roommates” Problem</a>) Algorithms and run in Colab these algorithms for those cases for n=4 and 40.<ul><li>Gale–Shapley algorithm simply explained – <a rel="noreferrer noopener" href="https://www.google.com/url?q=https%3A%2F%2Ftowardsdatascience.com%2Fgale-shapley-algorithm-simply-explained-caa344e643c2&amp;sa=D&amp;sntz=1&amp;usg=AFQjCNFY2_0Jta3gQm6u49dpS6SwA-b7Zw" target="_blank">towardsdatascience</a></li><li>Stable Marriage Problem – <a rel="noreferrer noopener" href="https://www.google.com/url?q=https%3A%2F%2Fwww.geeksforgeeks.org%2Fstable-marriage-problem%2F&amp;sa=D&amp;sntz=1&amp;usg=AFQjCNHOha_A_84VqRM1h4raycqrdD1yBw" target="_blank">geeksforgeeks</a></li><li>Stable matching, as a game – <a rel="noreferrer noopener" href="https://www.google.com/url?q=https%3A%2F%2Ftowardsdatascience.com%2Fstable-matching-as-a-game-a68c279d70b&amp;sa=D&amp;sntz=1&amp;usg=AFQjCNFzbKZ6J55Y5q-XkZffWF1nXvZOwA" target="_blank">towardsdatascience.com</a></li></ul>

In [24]:
%pylab inline
import pandas as pd
import numpy as np
from collections import Counter
from copy import copy

Populating the interactive namespace from numpy and matplotlib


## Gale-Shapley

### Mejor caso

El mejor caso para este algoritmo corresponde cuando cada hombre tiene como máxima prioridad a una mujer que lo tiene a él como máxima prioridad, mientras ningún otro hombre o mujer los tiene a ellos como prioridad máxima. Para este caso particular, las proposiciones de los hombres tendrían que ser idénticas en su respectivo DataFrame a la de las mujeres.

In [25]:
def women_proposals(n, index, women_dict, arr):
  initial_arr = [arr[0]] + arr[1:][::-1] #[1]+list(range(n, 1, -1))
  for i in range(n, 0, -1):
    name = "M" + str(n-i+1)
    arr = initial_arr[(i-n+1):] + initial_arr[:(i-n+1)]
    women_dict[name] = arr
    index.append("W"+str(n-i+1))
  return women_dict, index

In [26]:
def men_proposals(n, index, men_dict, arr):
  initial_arr = arr[:len(arr)-1] #list(range(1, n))
  for i in range(n-1, -1, -1):
    name = "M" + str(n-i)
    arr = initial_arr[i:] + initial_arr[:i] + [n]
    men_dict[name] = arr
    index.append("W"+str(n-i))
  return men_dict, index

En las celdas anteriores se encuentran las funciones generadoras de propuestas, tanto para hombres como mujeres. Dichas funciones se dan a partir de un arreglo semilla que debe ser una permutación de $n$ números con ningún elemento repetido (como en el laboratorio anterior). Las funciones anteriores se adaptaron para generar los peores casos, pero por ahora nos enfocaremos en el mejor caso, donde las relaciones de preferencia de hombres y mujeres son idénticas.

In [27]:
n = 4
arr = list(range(1, n+1)) # Para este caso, el arreglo semilla puede ser cualquiera
women_dict, women_index = women_proposals(n, [], {}, arr) 

# En el mejor caso se igualan en lugar de ejecutar la función correspondiente 
men_dict, men_index = women_dict, women_index 
man_list, women_list = men_dict.keys(), women_index

Esta es una función auxiliar para generar los dataframes. Debido a como se formaron las funciones, el DataFrame generado debe ser traspuesto para quedar con el mismo formato de las referencias.

In [28]:
def fill_dataframe(dictionary, index):
  gender = pd.DataFrame(dictionary)
  gender.index = index
  return gender.transpose()

In [29]:
man_df = fill_dataframe(men_dict, men_index)
man_df

,W1,W2,W3,W4
M1,4,3,2,1
M2,1,4,3,2
M3,2,1,4,3
M4,3,2,1,4


In [30]:
women_df = fill_dataframe(women_dict, women_index)
women_df

,W1,W2,W3,W4
M1,4,3,2,1
M2,1,4,3,2
M3,2,1,4,3
M4,3,2,1,4


Cabe aclarar que las personas se identifican por una clave alfanumérica. El primer símbolo corresponde a su género, siendo W para cuando es mujer y M para cuando es hombre. A continuación, le precede un número cualquiera entre $1$ y $n$.

In [31]:
def gale_shapley(waiting_list, man_list, women_available, man_df, women_df, proposals):
  # variable to count number of iterations
  count = 0
  # while not all men have pairs
  while len(waiting_list)<len(man_list):
    # man makes proposals
    for man in man_list:
        if man not in waiting_list:
            # each man make proposal to the top women from it's list
            women = women_available[man]
            best_choice = man_df.loc[man][man_df.loc[man].index.isin(women)].idxmin()
            proposals[(man, best_choice)] = (man_df.loc[man][best_choice], women_df.loc[man][best_choice])

    # if women have more than one proposals 
    # she will choose the best option
    overlays = Counter([key[1] for key in proposals.keys()])
    # cycle to choose the best options
    for women in overlays.keys():
        if overlays[women]>1:
            # pairs to drop from proposals
            pairs_to_drop = sorted({pair: proposals[pair] for pair in proposals.keys() 
                    if women in pair}.items(), 
                   key=lambda x: x[1][1]
                  )[1:]
            # if man was rejected by woman
            # there is no pint for him to make proposal 
            # second time to the same woman
            for p_to_drop in pairs_to_drop:
                del proposals[p_to_drop[0]]
                _women = copy(women_available[p_to_drop[0][0]])
                _women.remove(p_to_drop[0][1])
                women_available[p_to_drop[0][0]] = _women
    # man who successfully created pairs must be added to the waiting list 
    waiting_list = [man[0] for man in proposals.keys()]
    # update counter
    count+=1

  return count, proposals

La función anterior es la correspondiente al algoritmo de Gale-Shapley. Nótese que para dicha función hay que tener inicializadas varias variables, y por ello es importante el orden en las celdas de código.

In [32]:
# dict to control which women each man can make proposals
women_available = {man:women_list for man in man_list}
# waiting list of men that were able to create pair on each iteration
waiting_list = []
# dict to store created pairs
proposals = {}

count, proposals = gale_shapley(waiting_list, man_list, women_available, man_df, women_df, proposals)

El número de iteraciones en el mejor caso para $n=4$ es:

In [33]:
count

1

Las parejas ordenadas en el mejor caso para $n=4$ junto con el orden de prioridad que tenían cada una es:

In [34]:
proposals

{('M1', 'W4'): (1, 1),
 ('M2', 'W1'): (1, 1),
 ('M3', 'W2'): (1, 1),
 ('M4', 'W3'): (1, 1)}

Para $n=40$, este sería el algoritmo del mejor caso:

In [35]:
n = 40
arr = list(range(1, n+1)) # Para este caso, el arreglo semilla puede ser cualquiera
women_dict, women_index = women_proposals(n, [], {}, arr) 

# En el mejor caso se igualan en lugar de ejecutar la función correspondiente 
men_dict, men_index = women_dict, women_index 
man_list, women_list = men_dict.keys(), women_index

man_df = fill_dataframe(men_dict, men_index)
women_df = fill_dataframe(women_dict, women_index)

# dict to control which women each man can make proposals
women_available = {man:women_list for man in man_list}
# waiting list of men that were able to create pair on each iteration
waiting_list = []
# dict to store created pairs
proposals = {}

count, proposals = gale_shapley(waiting_list, man_list, women_available, man_df, women_df, proposals)

El número de iteraciones en el mejor caso para $n=40$ es:

In [36]:
count

1

Las parejas ordenadas en el mejor caso para $n=40$ junto con el orden de prioridad que tenían cada una es:

In [37]:
proposals

{('M1', 'W40'): (1, 1),
 ('M2', 'W1'): (1, 1),
 ('M3', 'W2'): (1, 1),
 ('M4', 'W3'): (1, 1),
 ('M5', 'W4'): (1, 1),
 ('M6', 'W5'): (1, 1),
 ('M7', 'W6'): (1, 1),
 ('M8', 'W7'): (1, 1),
 ('M9', 'W8'): (1, 1),
 ('M10', 'W9'): (1, 1),
 ('M11', 'W10'): (1, 1),
 ('M12', 'W11'): (1, 1),
 ('M13', 'W12'): (1, 1),
 ('M14', 'W13'): (1, 1),
 ('M15', 'W14'): (1, 1),
 ('M16', 'W15'): (1, 1),
 ('M17', 'W16'): (1, 1),
 ('M18', 'W17'): (1, 1),
 ('M19', 'W18'): (1, 1),
 ('M20', 'W19'): (1, 1),
 ('M21', 'W20'): (1, 1),
 ('M22', 'W21'): (1, 1),
 ('M23', 'W22'): (1, 1),
 ('M24', 'W23'): (1, 1),
 ('M25', 'W24'): (1, 1),
 ('M26', 'W25'): (1, 1),
 ('M27', 'W26'): (1, 1),
 ('M28', 'W27'): (1, 1),
 ('M29', 'W28'): (1, 1),
 ('M30', 'W29'): (1, 1),
 ('M31', 'W30'): (1, 1),
 ('M32', 'W31'): (1, 1),
 ('M33', 'W32'): (1, 1),
 ('M34', 'W33'): (1, 1),
 ('M35', 'W34'): (1, 1),
 ('M36', 'W35'): (1, 1),
 ('M37', 'W36'): (1, 1),
 ('M38', 'W37'): (1, 1),
 ('M39', 'W38'): (1, 1),
 ('M40', 'W39'): (1, 1)}

### Peor caso

La fuente sobre la cual se tomó en cuenta el análisis del peor caso es este <a href=https://www.mdpi.com/1999-4893/6/2/371/htm> artículo</a>, en el cual se explica la lógica detrás del peor caso, además de algunas apreciaciones aparte de las referenciadas al inicio de este notebook. 

Las relaciones de preferencia en el peor caso para las mujeres se pueden entender de la siguiente manera:

\begin{pmatrix}
n & n-1 & n-2 & \dots & 1  \\ 
1 & n & n-1 & \dots & 2\\ 
2 & 1 & n & \dots  & 3\\ 
\dots & \dots & \dots & \dots & \dots \\ 
n-1 & n-2 & \dots & 1 & n 
\end{pmatrix}

Mientras tanto, las relaciones de preferencia en el peor caso para los hombres se pueden concebir así:

\begin{pmatrix}
1 & 2 & 3 & \dots & n\\ 
n-1 & 1 & 2 & \dots & n\\ 
n-2 & n-1 & 1 & \dots & n\\ 
\dots & \dots & \dots & \dots & n\\ 
1 & 2 & 3 & \dots & n
\end{pmatrix}

Tanto la función de generación de parejas para hombres como para mujeres está diseñada sobre todo para el peor caso, siempre y cuando el arreglo semilla es el de una permutación ordenada de $n$ números. Para ver el peor caso con $n=4$ haremos lo mismo que antes, pero esta vez el DataFrame de las relaciones de preferencia de los hombres no se iguala al de las mujeres, sino que se crea a partir de la función de generación con el arreglo semilla de la permutación ordenada.

In [38]:
n = 4
arr = list(range(1, n+1)) 
women_dict, women_index = women_proposals(n, [], {}, arr)  
men_dict, men_index = men_proposals(n, [], {}, arr)
man_list, women_list = men_dict.keys(), women_index

In [39]:
man_df = fill_dataframe(men_dict, men_index)
man_df

,W1,W2,W3,W4
M1,1,2,3,4
M2,3,1,2,4
M3,2,3,1,4
M4,1,2,3,4


In [40]:
women_df = fill_dataframe(women_dict, women_index)
women_df

,W1,W2,W3,W4
M1,4,3,2,1
M2,1,4,3,2
M3,2,1,4,3
M4,3,2,1,4


In [41]:
# dict to control which women each man can make proposals
women_available = {man:women_list for man in man_list}
# waiting list of men that were able to create pair on each iteration
waiting_list = []
# dict to store created pairs
proposals = {}

count, proposals = gale_shapley(waiting_list, man_list, women_available, man_df, women_df, proposals)

El número de iteraciones en el peor caso para $n=4$ es:

In [42]:
count

10

Las parejas ordenadas en el peor caso para $n=4$ junto con el orden de prioridad que tenían cada una es:

In [43]:
proposals

{('M2', 'W1'): (3, 1),
 ('M3', 'W2'): (3, 1),
 ('M4', 'W3'): (3, 1),
 ('M1', 'W4'): (4, 1)}

Para $n=40$, este sería el algoritmo del peor caso:

In [44]:
n = 40
arr = list(range(1, n+1)) # Para este caso, el arreglo semilla puede ser cualquiera
women_dict, women_index = women_proposals(n, [], {}, arr) 
men_dict, men_index = men_proposals(n, [], {}, arr)
man_list, women_list = men_dict.keys(), women_index

man_df = fill_dataframe(men_dict, men_index)
women_df = fill_dataframe(women_dict, women_index)

# dict to control which women each man can make proposals
women_available = {man:women_list for man in man_list}
# waiting list of men that were able to create pair on each iteration
waiting_list = []
# dict to store created pairs
proposals = {}

count, proposals = gale_shapley(waiting_list, man_list, women_available, man_df, women_df, proposals)

El número de iteraciones en el peor caso para $n=40$ es:

In [45]:
count

1522

Las parejas ordenadas en el peor caso para $n=40$ junto con el orden de prioridad que tenían cada una es:

In [46]:
proposals

{('M2', 'W1'): (39, 1),
 ('M3', 'W2'): (39, 1),
 ('M4', 'W3'): (39, 1),
 ('M5', 'W4'): (39, 1),
 ('M6', 'W5'): (39, 1),
 ('M7', 'W6'): (39, 1),
 ('M8', 'W7'): (39, 1),
 ('M9', 'W8'): (39, 1),
 ('M10', 'W9'): (39, 1),
 ('M11', 'W10'): (39, 1),
 ('M12', 'W11'): (39, 1),
 ('M13', 'W12'): (39, 1),
 ('M14', 'W13'): (39, 1),
 ('M15', 'W14'): (39, 1),
 ('M16', 'W15'): (39, 1),
 ('M17', 'W16'): (39, 1),
 ('M18', 'W17'): (39, 1),
 ('M19', 'W18'): (39, 1),
 ('M20', 'W19'): (39, 1),
 ('M21', 'W20'): (39, 1),
 ('M22', 'W21'): (39, 1),
 ('M23', 'W22'): (39, 1),
 ('M24', 'W23'): (39, 1),
 ('M25', 'W24'): (39, 1),
 ('M26', 'W25'): (39, 1),
 ('M27', 'W26'): (39, 1),
 ('M28', 'W27'): (39, 1),
 ('M29', 'W28'): (39, 1),
 ('M30', 'W29'): (39, 1),
 ('M31', 'W30'): (39, 1),
 ('M32', 'W31'): (39, 1),
 ('M33', 'W32'): (39, 1),
 ('M34', 'W33'): (39, 1),
 ('M35', 'W34'): (39, 1),
 ('M36', 'W35'): (39, 1),
 ('M37', 'W36'): (39, 1),
 ('M38', 'W37'): (39, 1),
 ('M39', 'W38'): (39, 1),
 ('M40', 'W39'): (39, 1),
 ('